# 光学シミュレーション

## 参考文献
- [光学機能性フィルムのシミュレーションによる設計と実験的検証](https://www.kobelco.co.jp/technology-review/pdf/65_2/050-053.pdf)
- [光学薄膜の分光特性シミュレーション](https://www.jstage.jst.go.jp/article/sfj1989/48/9/48_9_890/_pdf)
- [基礎物理定数|ウシオ電機](https://www.ushio.co.jp/jp/technology/glossary/material/attached_material_03.html)

## スネルの法則
$$
n_{0}\sin\theta_{0}=n_{1}\sin\theta_{1} \\
\theta_{1} = \sin^{-1}\left(\frac{n_{0}}{n_{1}}\sin\theta_{0}\right)
$$ 

In [1]:
# import
import numpy as np
from copy import copy

In [2]:
# code
def snell(n1,n0,theta):
    return np.arcsin(n0.real/n1.real*np.sin(theta))


In [3]:
# test
snell(1+1j,1,0)

0.0

## 光学アドミタンス
$$
\eta =
\left\{
    \begin{array}{ll}
        Y_{0}n\cos\phi & \text{s偏光} \\
        \displaystyle\frac{Y_{0}n}{\cos\phi} & \text{p偏光}
    \end{array}
\right.
$$
ここで$Y_{0}$は真空の光学アドミタンスであり
$$
Y_{0} =\sqrt{\frac{\epsilon_{0}}{\mu_{0}}}
$$
$\epsilon_{0}$:真空の誘電率、$\mu_{0}$:真空の透磁率である。



In [4]:
# Constants
EPS = 8.8541878128e-12
MU = 1.25663706212e-6
Y0 = np.sqrt(EPS/MU)

In [5]:
# code
def admittance(n1,n0,theta):
    phi = snell(n1,n0,theta)
    return (Y0*n1*np.cos(phi), Y0*n1/np.cos(phi))

In [6]:
# test
admittance(1.0,1.0,0.0*np.pi/180)==(Y0,Y0)

True

## 位相差
$$
2\delta = \frac{4\pi}{\lambda}nd\cos\phi
$$

In [7]:
# code
def phasedifference(n,d,phi,lm):
    return 2*np.pi*n*d*np.cos(phi)/lm

In [8]:
# test
phasedifference(2.0,125,0,500)

3.141592653589793

## 特性マトリクス
$$
M_{i} = \left(
\begin{array}{cc}
\cos\delta_{i} & i\eta_{i}^{-1}\sin\delta_{i} \\
i\eta_{i}\sin\delta_{i} & \cos\delta_{i}
\end{array}
\right)
$$


In [9]:
# code
def characteristic_matrix(n,d,n0,angle,lam):
    eta = admittance(n,n0,angle)
    phi = snell(n,n0,angle)
    delta = phasedifference(n,d,phi,lam)
    c = np.cos(delta)
    s = np.sin(delta)
    def _matrix(et):
        return (np.array([[c,1j*s/et],[1j*s*et,c]]))
    return tuple(map(_matrix, eta))

In [10]:
# test
characteristic_matrix(1.0,50,1.0,0,550)

(array([[0.84125353+0.00000000e+00j, 0.        +2.03675785e+02j],
        [0.        +1.43508711e-03j, 0.84125353+0.00000000e+00j]]),
 array([[0.84125353+0.00000000e+00j, 0.        +2.03675785e+02j],
        [0.        +1.43508711e-03j, 0.84125353+0.00000000e+00j]]))

## 反射
$$
R = \left|\frac{\eta_{0}B-C}{\eta_{0}B+C}\right|^{2}
$$

In [11]:
# code
def reflectance(param, eta0):
    def _reflectance(param, eta0):
        return (np.abs((eta0*param[0]-param[1])/(eta0*param[0]+param[1]))**2)[0]
    return tuple(map(_reflectance,param,eta0))

In [12]:
# test
eta0 = admittance(1.0,1.0,0)
eta = admittance(1.5,1.0,0)
param = (np.array([[1],[eta[0]]]),np.array([[1],[eta[1]]]))
ref= reflectance(param,eta0)
#np.isclose(ref,(0.04,0.04))
ref

(0.040000000000000015, 0.040000000000000015)

## マトリクス法
$$
\left(\begin{array}{c}
    B \\
    C
\end{array}\right) = 
\prod_{i=1}^{n}M_{i}
\left(\begin{array}{c}
    1 \\
    \eta_{s}
\end{array}\right)
$$

In [13]:
# code
def calc_matrix(ns,ds,n0,angle,lam):
    mat = (np.eye(2),np.eye(2))
    for n,d in zip(ns,ds):
        mat = tuple(map(lambda m1,m2:np.dot(m1,m2),mat,characteristic_matrix(n,d,n0,angle,lam)))
    return mat

In [14]:
# test
n0 = 1.0
n_sub = 1.5
ns = [1.0,1.5]
ds = [100,100]
theta = 0*np.pi/180
lam = 550

mat = calc_matrix(ns,ds,n0,theta,lam)

eta_sub= admittance(n_sub,n0,theta)
vec = (np.array([[1],[eta_sub[0]]]),np.array([[1],[eta_sub[1]]]))
param = tuple(map(lambda m,v:np.dot(m,v),mat,vec))

eta0= admittance(n0,n0,0.0)
reflectance(param,eta0)

(0.040000000000000036, 0.040000000000000036)

## 透過
$$
T = \frac{4\eta_{0}\Re(\eta_{s})}{\left|\eta_{0}B+C\right|^{2}}
$$

In [15]:
# code
def transmittance(param,eta0,eta1):
    def _transmittance(param,eta0,eta1):
        return (4*eta0.real*eta1.real/np.abs(eta0*param[0]+param[1])**2)[0]
    return tuple(map(_transmittance, param, eta0, eta1))

In [16]:
# test
n0 = 1.0
n_sub = 1.5
ns = [1.0,1.5]
ds = [100,100]
theta = 0*np.pi/180
lam = 550

mat = calc_matrix(ns,ds,n0,theta,lam)

eta_sub= admittance(n_sub,n0,theta)
vec = (np.array([[1],[eta_sub[0]]]),np.array([[1],[eta_sub[1]]]))
param = tuple(map(lambda m,v:np.dot(m,v),mat,vec))

eta0= admittance(n0,n0,0.0)
transmittance(param,eta0,eta_sub)

(0.9600000000000001, 0.9600000000000001)

## ベクトル化


In [17]:
def calc_matrix(ns,ds,n0,n1,theta,lam):
    length = len(lam)
    eta = admittance(n1,n0,theta)
    
    v_s = tuple([np.array([np.ones(length),eta[0]]),
                 np.array([np.ones(length),eta[1]])])
    
    mat = tuple([np.array([[np.ones(length),np.zeros(length)],[np.zeros(length),np.ones(length)]]),
                 np.array([[np.ones(length),np.zeros(length)],[np.zeros(length),np.ones(length)]])])
    for n,d in zip(ns,ds):
        m = characteristic_matrix(n, d, n0,theta,lam)
        mat = tuple([np.array([[mat[0][0,0]*m[0][0,0]+mat[0][0,1]*m[0][1,0], mat[0][0,0]*m[0][0,1]+mat[0][0,1]*m[0][1,1]],
                               [mat[0][1,0]*m[0][0,0]+mat[0][1,1]*m[0][1,0], mat[0][1,0]*m[0][0,1]+mat[0][1,1]*m[0][1,1]]]),
                     np.array([[mat[1][0,0]*m[1][0,0]+mat[1][0,1]*m[1][1,0], mat[1][0,0]*m[1][0,1]+mat[1][0,1]*m[1][1,1]],
                               [mat[1][1,0]*m[1][0,0]+mat[1][1,1]*m[1][1,0], mat[1][1,0]*m[1][0,1]+mat[1][1,1]*m[1][1,1]]])])

    param = tuple([np.array([[mat[0][0,0]*v_s[0][0]+mat[0][0,1]*v_s[0][1]],[mat[0][1,0]*v_s[0][0]+mat[0][1,1]*v_s[0][1]]]),
                   np.array([[mat[1][0,0]*v_s[1][0]+mat[1][0,1]*v_s[1][1]],[mat[1][1,0]*v_s[1][0]+mat[1][1,1]*v_s[1][1]]])])
    return param

In [18]:
n0 = 1.0
nl = np.array([1.5,1.5,1.5])
nh = np.array([2.3,2.3,2.3])
lam = np.array([500,550,600])
theta = 0*np.pi/180

ns = [nl,nh,nl,nh]
ds = [90,90,30,30]

eta0 = admittance(n0,n0,theta)
eta_s = admittance(nl,n0,theta)
param = calc_matrix(ns,ds,n0,nl,theta,lam)
print(reflectance(param,eta0))
print(transmittance(param,eta0,eta_s))

(array([0.03949384, 0.01877527, 0.02192589]), array([0.03949384, 0.01877527, 0.02192589]))
(array([0.96050616, 0.98122473, 0.97807411]), array([0.96050616, 0.98122473, 0.97807411]))


## 有限厚みの基板
<div align="center">

![反射](svg/reflect.drawio.svg)  

</div>

反射率は
$$
\begin{align*}
R &= r_{01}+t_{01}r_{12}t_{10}+t_{01}r_{12}r_{10}r_{12}t_{10}+t_{01}r_{12}r_{10}r_{12}r_{10}r_{12}t_{10}+\cdots \\
  &= r_{01}+t_{01}r_{12}t_{10}+t_{01}r_{12}t_{10}(r_{10}r_{12})+t_{01}r_{12}t_{10}(r_{10}r_{12})^{2}+\cdots \\
  &= r_{01}+\sum_{n=1}^{\infty}t_{01}r_{12}t_{10}(r_{10}r_{12})^{n-1} \\
  &= r_{01}+\frac{t_{01}r_{12}t_{10}}{1-r_{10}r_{12}}
\end{align*}
$$
同様に透過率は
$$
\begin{align*}
T &= t_{01}t_{12}+t_{01}r_{12}r_{10}t_{12}+t_{01}r_{12}r_{10}r_{12}r_{10}t_{12}+\cdots \\
  &= t_{01}t_{12}+t_{01}t_{12}(r_{12}r_{10})+t_{01}t_{12}(r_{12}r_{10})^{2}+\cdots \\
  &= \sum_{n=1}^{\infty}t_{01}t_{12}(r_{12}r_{10})^{n-1} \\
  &= \frac{t_{01}t_{12}}{1-r_{12}r_{10}}

\end{align*}
$$
$$
r_{ij}=\left|\frac{\eta_{i}-\eta_{j}}{\eta_{i}+\eta_{j}}\right|^{2} \\
r_{ij}+t_{ij} = 1
$$
なので
$$
r_{ij} = r_{ji} \\
t_{ij} = 1- r_{ij}
$$
結果として反射率は
$$
R = r_{01}+\frac{(1-r_{01})^{2}r_{12}}{1-{r_{01}r_{12}}} = \frac{r_{01}+r_{12}-2r_{01}r_{12}}{1-r_{01}r_{12}}
$$
透過率は
$$
T = \frac{1-(r_{01}+r_{12})+r_{01}r_{12}}{1-r_{01}r_{12}}
$$

In [19]:
n0 = 1.0
n1 = np.array([1.6,1.55,1.50])
n2 = 1.0
theta = 0*np.pi/180
eta0 = admittance(n0,n0,theta)
eta1 = admittance(n1,n0,theta)
eta2 = admittance(n2,n0,theta)
r01 = (np.abs((eta0[0]-eta1[0])/(eta0[0]+eta1[0]))**2, np.abs((eta0[1]-eta1[1])/(eta0[1]+eta1[1]))**2)
r12 = (np.abs((eta1[0]-eta2[0])/(eta1[0]+eta2[0]))**2, np.abs((eta1[1]-eta2[1])/(eta1[1]+eta2[1]))**2)
r = ((r01[0]+r12[0]-2*r01[0]*r12[0])/(1-r01[0]*r12[0]),(r01[1]+r12[1]-2*r01[1]*r12[1])/(1-r01[1]*r12[1]))
t = ((1-(r01[0]+r12[0])+2*r01[0]*r12[0])/(1-r01[0]*r12[0]),(1-(r01[1]+r12[1])+2*r01[1]*r12[1])/(1-r01[1]*r12[1]))
print(r)
print(t)

(array([0.1011236 , 0.08890522, 0.07692308]), array([0.1011236 , 0.08890522, 0.07692308]))
(array([0.90172051, 0.91326364, 0.92467949]), array([0.90172051, 0.91326364, 0.92467949]))


## 有限厚みの基板（吸収を考慮）
<div align="center">

![反射](svg/reflect.drawio.svg)  

</div>

光路長は
$$
l = \frac{2d}{\cos\phi}
$$
吸収係数は
$$
\alpha = \frac{8\pi k}{\lambda}
$$
なので
$$
I = I_{0}e^{-i\alpha l} = I_{0}\exp\left(-\frac{8\pi kd}{\lambda\cos\phi}\right)
$$
この時
$$
\beta = \exp\left(-\frac{8\pi kd}{\lambda\cos\phi}\right)
$$
とすると、反射率は
$$
\begin{align*}
R &= r_{01}+t_{01}r_{12}t_{10}\beta+t_{01}r_{12}r_{10}r_{12}t_{10}\beta^{2}+t_{01}r_{12}r_{10}r_{12}r_{10}r_{12}t_{10}\beta^{3}+\cdots \\
  &= r_{01}+t_{01}r_{12}t_{10}\beta+t_{01}r_{12}t_{10}\beta(r_{10}r_{12}\beta)+t_{01}r_{12}t_{10}\beta(r_{10}r_{12}\beta)^{2}+\cdots \\
  &= r_{01}+\sum_{n=1}^{\infty}t_{01}r_{12}t_{10}\beta(r_{10}r_{12}\beta)^{n-1} \\
  &= r_{01}+\frac{t_{01}r_{12}t_{10}\beta}{1-r_{10}r_{12}\beta}
\end{align*}
$$
同様に透過率は
$$
\begin{align*}
T &= t_{01}t_{12}\beta^{1/2}+t_{01}r_{12}r_{10}t_{12}\beta^{3/2}+t_{01}r_{12}r_{10}r_{12}r_{10}t_{12}\beta^{5/2}+\cdots \\
  &= t_{01}t_{12}\beta^{1/2}+t_{01}t_{12}\beta^{1/2}(r_{12}r_{10}\beta)+t_{01}t_{12}\beta^{1/2}(r_{12}r_{10}\beta)^{2}+\cdots \\
  &= \sum_{n=1}^{\infty}t_{01}t_{12}\beta^{1/2}(r_{12}r_{10}\beta)^{n-1} \\
  &= \frac{t_{01}t_{12}\beta^{1/2}}{1-r_{12}r_{10}\beta}
\end{align*}
$$
吸収が無い場合と同様に考えると反射率は
$$
R = r_{01}+\frac{(1-r_{01})^{2}r_{12}\beta}{1-{r_{01}r_{12}\beta}} = \frac{r_{01}+r_{12}\beta-2r_{01}r_{12}\beta}{1-r_{01}r_{12}\beta}
$$
透過率は
$$
T = \frac{(1-(r_{01}+r_{12})+r_{01}r_{12})\beta^{1/2}}{1-r_{01}r_{12}\beta}
$$

In [20]:
def absorb(n1,d,n0,theta,lam):
    phi = snell(n1,n0,theta)
    return np.exp(-8*np.pi*np.abs(n1.imag)*d*1000/(lam*np.cos(phi)))

In [21]:
n0 = 1.0
n1 = 1.6+0.000001j
n2 = 1.0
theta = 85*np.pi/180
d = 100
lam = 550
beta = absorb(n1,d,n0,theta,lam)

eta0 = admittance(n0,n0,theta)
eta1 = admittance(n1,n0,theta)
eta2 = admittance(n2,n0,theta)
r01 = (np.abs((eta0[0]-eta1[0])/(eta0[0]+eta1[0]))**2, np.abs((eta0[1]-eta1[1])/(eta0[1]+eta1[1]))**2)
r12 = (np.abs((eta1[0]-eta2[0])/(eta1[0]+eta2[0]))**2, np.abs((eta1[1]-eta2[1])/(eta1[1]+eta2[1]))**2)
r = ((r01[0]+r12[0]*beta-2*r01[0]*r12[0]*beta)/(1-r01[0]*r12[0]*beta),
     (r01[1]+r12[1]*beta-2*r01[1]*r12[1]*beta)/(1-r01[1]*r12[1]*beta))
t = ((1-(r01[0]+r12[0])+2*r01[0]*r12[0])*np.sqrt(beta)/(1-r01[0]*r12[0]*beta),
     (1-(r01[1]+r12[1])+2*r01[1]*r12[1])*np.sqrt(beta)/(1-r01[1]*r12[1]*beta))
print(r)
print(t)
print(beta)
print(r01[1]-r12[1]*beta)

(0.8600323956870074, 0.6532812621599582)
(1.461873501203525, 0.6524263532069948)
0.9941774579275747
0.0028326873245643203


## 基板に薄膜がついている場合
基板のみの場合から
$$
R = r_{01}+\frac{t_{01}r_{12}t_{10}\beta}{1-r_{10}r_{12}\beta} \\[10pt]
T = \frac{t_{01}t_{12}\beta^{1/2}}{1-r_{10}r_{12}\beta}
$$

それぞれ  
<div align="center">

|表記|入射|多層膜|出射|
|-|-|-|-|
|$r_{01}$|媒質0|表面側（正順）|媒質1|
|$r_{10}$|媒質1|表面側（逆順）|媒質0|
|$r_{12}$|媒質1|裏面側（逆順）|媒質2|
|$t_{01}$|媒質0|表面側（正順）|媒質1|
|$t_{10}$|媒質1|表面側（逆順）|媒質0|
|$t_{12}$|媒質1|裏面側（逆順）|媒質2|

</div>

In [22]:
def calc_spectra(n0,n1,n2,theta,front_ns,front_ds,back_ns,back_ds,t_sub, lam):
     f_ns = copy(front_ns)
     f_ds = copy(front_ds)
     b_ns = copy(back_ns)
     b_ds = copy(back_ds)
     
     phi1 = snell(n1,n0,theta)
     # phi2 = snell(n2,n0,theta)
    
     eta0 = admittance(n0,n0,theta)
     eta1 = admittance(n1,n0,theta)
     eta2 = admittance(n2,n0,theta)
     
     front_forward = calc_matrix(f_ns,f_ds,n0,n1,theta,lam)
     f_ns.reverse()
     f_ds.reverse()
     front_backward = calc_matrix(f_ns,f_ds,n1,n0,phi1,lam)
     
     # back_forward = calc_matrix(b_ns,b_ds,n2,n1,phi2,lam)
     b_ns.reverse()
     b_ds.reverse()
     back_backward = calc_matrix(b_ns,b_ds,n1,n2,phi1,lam)
     
     r01 = reflectance(front_forward,eta0)
     r10 = reflectance(front_backward,eta1)
     t01 = transmittance(front_forward,eta0,eta1)
     t10 = transmittance(front_backward,eta1,eta0)
     
     r12 = reflectance(back_backward,eta1)
     t12 = transmittance(back_backward,eta1,eta2) 

     beta = absorb(n1,t_sub,n0,theta,lam)

     r = (r01[0]+ (t01[0]*r12[0]*t10[0]*beta)/(1-r10[0]*r12[0]*beta), 
          r01[1]+ (t01[1]*r12[1]*t10[1]*beta)/(1-r10[1]*r12[1]*beta))
     t = (t01[0]*t12[0]*np.sqrt(beta)/(1-r10[0]*r12[0]*beta),
          t01[1]*t12[1]*np.sqrt(beta)/(1-r10[1]*r12[1]*beta))

     return (r01,t01,r,t)

In [23]:
n0 = 1.0
n1 = np.array([1.5-1e-6j,1.5,1.5])
n2 = 1.0
nl = np.array([1.5,1.5,1.5])
nh = np.array([2.3,2.3,2.3])
lam = np.array([500,550,600])

theta = 85*np.pi/180

t_sub = 100

front_ns = [nl,nh,nl,nh]
front_ds = [90,90,30,30]

back_ns = [nl,nh,nl,nh]
back_ds = [90,90,30,30]



In [24]:
calc_spectra(n0,n1,n2,theta,front_ns,front_ds,back_ns,back_ds, t_sub, lam)


((array([0.67677811, 0.71615291, 0.70423883]),
  array([0.55704801, 0.52788084, 0.52948574])),
 (array([0.32322189, 0.28384709, 0.29576117]),
  array([0.44295199, 0.47211916, 0.47051426])),
 (array([0.80563259, 0.83460268, 0.82645556]),
  array([0.71398332, 0.69099739, 0.69237094])),
 (array([0.19103503, 0.16539732, 0.17354444]),
  array([0.28267517, 0.30900261, 0.30762906])))

In [25]:
import opticalsimulation.opticalsimulation as op

In [26]:
op.calc_spectra(n0,n1,n2,theta,front_ns,front_ds,back_ns,back_ds, t_sub, lam)


((array([0.67677811, 0.71615291, 0.70423883]),
  array([0.55704801, 0.52788084, 0.52948574])),
 (array([0.32322189, 0.28384709, 0.29576117]),
  array([0.44295199, 0.47211916, 0.47051426])),
 (array([0.80563259, 0.83460268, 0.82645556]),
  array([0.71398332, 0.69099739, 0.69237094])),
 (array([0.19103503, 0.16539732, 0.17354444]),
  array([0.28267517, 0.30900261, 0.30762906])))